In [21]:
from pandas import *
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.22 ms


In [22]:
data_path = '../boots_dataset/'

time: 516 µs


In [23]:
metadata = read_csv(data_path + 'boots_aws_160403.csv', index_col = 'Unnamed: 0').drop_duplicates()
metadata.index.name = '#'

time: 142 ms


In [24]:
metadata.head()

,title,brand,price,asin,related,imUrl,salesRank,categories,description,pic_downloaded,boots,sex,image_path
#,,,,,,,,,,,,,
1139,Justin Boots Black Iguana Lizard,NaN,329.95,B0000A1R6F,NaN,http://ecx.images-amazon.com/images/I/51AutsCt...,{'Shoes': 249279},NaN,NaN,True,True,unknown,pics_boots/B0000A1R6F.jpg
1141,"""Justin Boots Men's Classic Western Boot""",NaN,NaN,B0000A1RDO,"{'also_bought': ['B000E9626I','B0041T42DC']'al...",http://ecx.images-amazon.com/images/I/41zrtA71...,{'Shoes': 53027},NaN,NaN,True,True,men,pics_boots/B0000A1RDO.jpg
1142,"""Justin Boots Men's Classic Western Boot""",NaN,NaN,B0000A1REG,"{'also_bought': ['B000E9626I','B0041T42DC']'al...",http://ecx.images-amazon.com/images/I/41zrtA71...,{'Shoes': 53027},NaN,NaN,True,True,men,pics_boots/B0000A1REG.jpg
2134,"""Children's Ostrich-Print Cowboy Boots-peanut ...",NaN,NaN,B0000CH8HL,"{'also_bought': ['B00551RXJK','B002LMHSEG','B0...",http://ecx.images-amazon.com/images/I/21M7uJMJ...,{'Shoes': 89070},NaN,NaN,True,True,men,pics_boots/B0000CH8HL.jpg
2785,"""Men's Itasca Waterproof Swampwalker II Rubber...",NaN,NaN,B0000WYVSA,"{'also_viewed': ['B002OSXDSC','B002BH4GAU','B0...",http://ecx.images-amazon.com/images/I/41S6OxKc...,{'Shoes': 89433},NaN,NaN,True,True,men,pics_boots/B0000WYVSA.jpg


time: 26.8 ms


In [25]:
keys = ['also_viewed','also_bought','bought_together','buy_after_viewing']

def extract_key_value(kv, item_dict):
    kv_split = kv.split(": ")
    k = kv_split[0][1:].replace("'",'')
    v = ": ".join(kv_split[1:]).replace("''","','")
    if v[0] == "'":
        v = v[1:-1]
    item_dict[k] = v
    return item_dict

def make_item_dict(item):
    item_dict = {}
    curr = ''

    for i in item:
        key_found = [k for k in keys if k in i]
        if len(key_found) == 1:
            if curr != '':
                item_dict = extract_key_value(curr, item_dict)
            curr = i
        elif len(key_found) == 0:
            curr += i
        else:
            print 'TOO MANY KEYS'
            print i
            print item
            sys.exit(0)
    if curr != '':
        item_dict = extract_key_value(curr, item_dict)
    return item_dict

time: 24 ms


In [26]:
def extract_related(x):
    if type(x) is float:
        return x
    x = x[1:-2].replace("']'","'], '").split(', ')
    return make_item_dict(x)
    
metadata.loc[:, 'related_d'] = metadata.loc[:, 'related'].apply(lambda x: extract_related(x))

time: 170 ms


In [27]:
def extract_dict(x, k):
    if type(x) is float:
        return x
    if k in x.keys():
        return x[k].replace('[','').replace(']','').split(',')
    else:
        return float('nan')

for k in keys:
    metadata.loc[:, k] = metadata.loc[:, 'related_d']\
                                 .apply(lambda x: extract_dict(x, k))

metadata = metadata.drop('related_d', axis = 1)

time: 148 ms


In [28]:
metadata.head()

,title,brand,price,asin,related,imUrl,salesRank,categories,description,pic_downloaded,boots,sex,image_path,also_viewed,also_bought,bought_together,buy_after_viewing
#,,,,,,,,,,,,,,,,,
1139,Justin Boots Black Iguana Lizard,NaN,329.95,B0000A1R6F,NaN,http://ecx.images-amazon.com/images/I/51AutsCt...,{'Shoes': 249279},NaN,NaN,True,True,unknown,pics_boots/B0000A1R6F.jpg,NaN,NaN,NaN,NaN
1141,"""Justin Boots Men's Classic Western Boot""",NaN,NaN,B0000A1RDO,"{'also_bought': ['B000E9626I','B0041T42DC']'al...",http://ecx.images-amazon.com/images/I/41zrtA71...,{'Shoes': 53027},NaN,NaN,True,True,men,pics_boots/B0000A1RDO.jpg,"['B00GJOPGNC', 'B0051SFB4Q', 'B000YDCUW4', 'B0...","['B000E9626I', 'B0041T42DC']",NaN,NaN
1142,"""Justin Boots Men's Classic Western Boot""",NaN,NaN,B0000A1REG,"{'also_bought': ['B000E9626I','B0041T42DC']'al...",http://ecx.images-amazon.com/images/I/41zrtA71...,{'Shoes': 53027},NaN,NaN,True,True,men,pics_boots/B0000A1REG.jpg,"['B00GJOPGNC', 'B0051SFB4Q', 'B000YDCUW4', 'B0...","['B000E9626I', 'B0041T42DC']",NaN,NaN
2134,"""Children's Ostrich-Print Cowboy Boots-peanut ...",NaN,NaN,B0000CH8HL,"{'also_bought': ['B00551RXJK','B002LMHSEG','B0...",http://ecx.images-amazon.com/images/I/21M7uJMJ...,{'Shoes': 89070},NaN,NaN,True,True,men,pics_boots/B0000CH8HL.jpg,NaN,"['B00551RXJK', 'B002LMHSEG', 'B0007UO42S', 'B0...","['B00551RXJK', 'B002LMHSEG', 'B0007UO42S', 'B0...",NaN
2785,"""Men's Itasca Waterproof Swampwalker II Rubber...",NaN,NaN,B0000WYVSA,"{'also_viewed': ['B002OSXDSC','B002BH4GAU','B0...",http://ecx.images-amazon.com/images/I/41S6OxKc...,{'Shoes': 89433},NaN,NaN,True,True,men,pics_boots/B0000WYVSA.jpg,"['B002OSXDSC', 'B002BH4GAU', 'B000HHQE74', 'B0...",NaN,NaN,NaN


time: 36.7 ms


In [29]:
def sales_rank(x, val):
    if type(x) is float or x == '{}':
        return float('nan')
    x = x.replace('{','').replace('}','').replace("'",'').split(':')
    if val == 'cat':
        return x[0].strip()
    else:
        return x[1].strip()

metadata.loc[:, 'salesRank_category'] = metadata.loc[:, 'salesRank'].apply(lambda x: sales_rank(x, 'cat'))
metadata.loc[:, 'salesRank_rank'] = metadata.loc[:, 'salesRank'].apply(lambda x: sales_rank(x, 'rank'))

time: 68.5 ms


In [30]:
metadata.to_csv(data_path + 'boots_aws_metadata_160405.csv')

time: 465 ms
